In [0]:
#Imports
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp, when, col

#configuration
spark.conf.set(
  "fs.azure.account.key.ecommercestorage5400.dfs.core.windows.net",
    "<storage_account_access_key>"
)

#src path
bronze_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/bronze"
#dest path
silver_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/silver"

df_bronze = (
    spark.readStream.
    format("delta")
    .load(bronze_path)
)

#cleaned data (Here produced data using faker so not much cleaning required)
df_clean = (
    df_bronze
    .withColumn("timestamp",to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
    .withColumn('price',when(col('price').isNull(), 0.0).otherwise(col('price')))
    .withColumn('quantity', when(col('quantity').isNull(), 0).otherwise(col('quantity')))
    .withColumn('total_amount', col('price')*col('quantity'))
    .dropDuplicates(['order_id'])
    .filter(col('country')=='USA')
    .filter(col('state').isNotNull())
)

# Write to silver layer
( df_clean.writeStream
  .format("delta")
  .option("checkpointLocation", f'{silver_path}/_checkpoint')
  .start(silver_path)
)

In [0]:
df_silver=spark.read.format('delta').load(silver_path)
display(df_silver.limit(10))

order_id,timestamp,customer_id,product_id,category,price,quantity,total_amount,city,state,country,latitude,longitude,delivery_status
98d4aa56-1942-469d-9881-849b0d67abc8,2025-11-25T05:08:46.834868Z,736a0386-1a71-412d-81f2-e406ec28457d,bce3a343-18ac-45bf-99b9-4316c481c016,Home Decor,1101.35,1,1101.35,Phoenix,AZ,USA,33.4484,-112.074,Shipped
354c0114-2607-431f-bc45-b03eb644bbfe,2025-11-25T05:10:00.888032Z,e08ee24a-3921-441c-8b1e-174e2880fa6b,d5f212a3-0afa-4dc6-86a7-53e75471a088,Home Decor,504.67,2,1009.34,Phoenix,AZ,USA,33.4484,-112.074,Processing
e514b16e-0c95-4120-9edb-7c3e862d86d4,2025-11-25T05:12:59.056514Z,bda21438-b79a-4559-844f-769b5b8e03fd,288b159b-d396-43fa-8748-7c7d1591da11,Books,1094.62,3,3283.8599999999997,Phoenix,AZ,USA,33.4484,-112.074,Shipped
c7aaa30c-d03e-4d59-b355-e1f00534cef7,2025-11-25T04:42:41.417767Z,2f7213b5-4f53-4381-8808-1fd106230a96,e85a803f-f658-48ec-9025-208ceaa66f08,Clothing,1950.57,1,1950.57,Los Angeles,CA,USA,34.0522,-118.2437,Processing
a0deada2-a46e-4021-ac7a-c2206a22cf58,2025-11-25T04:53:21.905731Z,706b406c-7909-4b6b-9b70-a84a8e463332,428e525a-a416-4014-beaa-0b599daad0e3,Electronics,463.3,2,926.6,Houston,TX,USA,29.7604,-95.3698,Cancelled
8b5b6260-6247-4565-b418-f5cc356b8f82,2025-11-25T05:00:52.290369Z,0485fbfc-4936-4e56-95d2-c5c62c20a631,3f3c15b2-66a4-4612-bdc2-41ba661aa258,Home Decor,1717.26,2,3434.52,Chicago,IL,USA,41.8781,-87.6298,Processing
ebf62efd-f7db-439d-b144-ebfbfe816788,2025-11-25T05:14:13.123256Z,bd4c68ff-4c4f-4ec5-aedf-c21e29c03e73,5108ab06-dbee-4699-914a-8000efe2b6f9,Clothing,1322.3,3,3966.8999999999996,Houston,TX,USA,29.7604,-95.3698,Processing
41d1ab2c-dcb2-408b-b2a4-4746a9872692,2025-11-25T05:02:30.383427Z,325bc280-b23e-41e2-8950-0195940d5d8a,5d31eb5c-b2b1-480c-938a-53f3f943a76a,Clothing,1284.37,4,5137.48,New York,NY,USA,40.7128,-74.006,Shipped
4e6e0547-d9c3-4a67-8da8-3ee7530ea820,2025-11-25T04:11:41.007527Z,095876f8-8d0b-4332-ba06-c4f03c7f3d31,b8d190d5-0f48-4e9f-9a85-89779b312924,Electronics,844.02,1,844.02,New York,NY,USA,40.7128,-74.006,Processing
ad689cfc-8581-45d9-a8f8-2245ae32eefa,2025-11-25T04:21:22.598764Z,aa8a5e52-757e-477d-b4eb-6c796780907f,fe18b127-5493-45f6-a902-f27b97d171d3,Home Decor,1402.15,4,5608.6,New York,NY,USA,40.7128,-74.006,Cancelled
